In [25]:
#from img2vec_keras import Img2Vec
from img2vec_pytorch import Img2Vec
from PIL import Image
import numpy as np
import cv2 as cv
import os
import pandas as pd
import warnings
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
warnings.filterwarnings('ignore')

In [26]:
dataset='D:/dataset2'

In [27]:
category = ['cloudy', 'sunrise', 'shine', 'rain']
three, two = 0, 0
for ind in category:
    category_path = os.path.join(dataset, ind)
    for file in os.listdir(category_path):
        path = os.path.join(category_path, file)
        img = cv.imread(path)
        if img is not None:
            shape = img.shape
            if len(shape) == 2:
                two += 1
            elif len(shape) == 3:
                three += 1
        else:
            print(f"Failed to load image: {path}")
print(two)
print(three)

0
1123


In [28]:
from PIL import Image
data,labels=[],[]
img2vec = Img2Vec()
category = ['cloudy', 'rain','shine','sunrise']
for ind,ele in enumerate(category):
    category_path = os.path.join(dataset, ele)
    for file in os.listdir(category_path):
        path = os.path.join(category_path, file)
        img = Image.open(path).convert('RGB')
        if img is not None:
            d=img2vec.get_vec(img)
            #print(d)
            data.append(d)
            labels.append(ind)
        else:
            print(f"Failed to load image: {path}")

In [29]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(data,labels,test_size=0.3,random_state=1,stratify=labels)

In [30]:
from sklearn.ensemble import RandomForestClassifier
classifer=RandomForestClassifier(n_estimators=50,max_depth=5,oob_score=True)

In [31]:
classifer.fit(xtrain,ytrain)
ypred=classifer.predict(xtest)

In [32]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
print(accuracy_score(ytest,ypred))
#print(precision_score(ytest,ypred))
#print(recall_score(ytest,ypred))
#print(f1_score(ytest,ypred))

0.9703264094955489


In [33]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier,OneVsRestClassifier
model=SVC(kernel='poly')
classifier_svm=OneVsOneClassifier(model)

In [34]:
classifier_svm.fit(xtrain,ytrain)
ypred1=classifier_svm.predict(xtest)

In [35]:
model.fit(xtrain,ytrain)
ypred=model.predict(xtest)

In [36]:
accuracy_score(ytest,ypred)

0.9940652818991098

In [37]:
accuracy_score(ytest,ypred1)

0.9881305637982196

In [79]:
data_url='D:/dataset2/*/*'
data_url
data=tf.data.Dataset.list_files(data_url,shuffle=True)
size=int(len(data)*0.8)
train_ds=data.take(size)
test_ds=data.skip(size)
print(len(train_ds))
print(len(test_ds))
for i in train_ds.take(2).as_numpy_iterator():
    print(tf.strings.split(i,'\\')[-2])
def get_label(filepath):
    return tf.strings.split(filepath,'\\')[-2]
def preprocess_img(filepath):
    img=tf.io.read_file(filepath)
    label=get_label(filepath)
    img=tf.io.decode_jpeg(img,channels=3)
    img = tf.image.resize(img, [160, 160])         
    img = img / 255.0 
    return img,label
for i in train_ds.take(10).map(get_label,num_parallel_calls=tf.data.AUTOTUNE):
    print(i)
for img,label in train_ds.map(preprocess_img,num_parallel_calls=tf.data.AUTOTUNE).take(1):
    print(img.dtype)
    print(label)


898
225
tf.Tensor(b'cloudy', shape=(), dtype=string)
tf.Tensor(b'shine', shape=(), dtype=string)
tf.Tensor(b'cloudy', shape=(), dtype=string)
tf.Tensor(b'cloudy', shape=(), dtype=string)
tf.Tensor(b'cloudy', shape=(), dtype=string)
tf.Tensor(b'cloudy', shape=(), dtype=string)
tf.Tensor(b'sunrise', shape=(), dtype=string)
tf.Tensor(b'cloudy', shape=(), dtype=string)
tf.Tensor(b'shine', shape=(), dtype=string)
tf.Tensor(b'shine', shape=(), dtype=string)
tf.Tensor(b'cloudy', shape=(), dtype=string)
tf.Tensor(b'sunrise', shape=(), dtype=string)
<dtype: 'float32'>
tf.Tensor(b'rain', shape=(), dtype=string)


In [82]:
# Extract class names from your dataset
class_names = sorted(set([str(tf.strings.split(i, '\\')[-2].numpy(), 'utf-8')for i in data.as_numpy_iterator()]))

print(class_names) 
label_to_index = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(class_names),
        values=tf.constant(list(range(len(class_names))), dtype=tf.int64),
    ),
    default_value=-1  
)
def get_label(filepath):
    class_name = tf.strings.split(filepath, '\\')[-2]
    return label_to_index.lookup(class_name)

def preprocess_img(filepath):
    label = get_label(filepath)
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [160, 160])
    img = img / 255.0
    return img, label
train_ds = train_ds.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # accepts integer labels
    metrics=['accuracy']
)

model.fit(train_ds, validation_data=test_ds, epochs=35)

['cloudy', 'rain', 'shine', 'sunrise']
Epoch 1/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 15s 464ms/step - accuracy: 0.3457 - loss: 30.4179 - val_accuracy: 0.6711 - val_loss: 6.4841
Epoch 2/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 452ms/step - accuracy: 0.6718 - loss: 4.2824 - val_accuracy: 0.5111 - val_loss: 5.1715
Epoch 3/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 464ms/step - accuracy: 0.6357 - loss: 3.2311 - val_accuracy: 0.6267 - val_loss: 5.8511
Epoch 4/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 14s 483ms/step - accuracy: 0.5894 - loss: 6.4596 - val_accuracy: 0.8222 - val_loss: 0.7384
Epoch 5/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 423ms/step - accuracy: 0.7689 - loss: 1.0132 - val_accuracy: 0.5956 - val_loss: 1.8225
Epoch 6/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 420ms/step - accuracy: 0.6724 - loss: 1.3835 - val_accuracy: 0.6889 - val_loss: 1.3576
Epoch 7/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 443ms/step - accuracy: 0.7879 - loss: 0.7564 - val_accuracy: 0.7822 - val_loss: 0.5674
Epoch 8/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 413ms/step - ac

In [83]:
model.evaluate(test_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9206 - loss: 0.2387


[0.27034735679626465, 0.897777795791626]